<a href="https://colab.research.google.com/github/anindyaroy/learnAgenticAI/blob/main/Translation_App_LanChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the necessary libraries for LangChain and FastAPI

In [ ]:
!pip install langchain
!pip install -qU langchain-openai
!pip install fastapi
!pip install uvicorn
!pip install langserve[all]

import the necessary modules and input your OpenAI API key

In [ ]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your OpenAI API Key:')

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")

Using the Model for English to Japanese Translation
We will now define a system message to specify the translation task (English to Japanese) and a human message to input the text to be translated.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage


messages = [
    SystemMessage(content="Translate the following from English into Japanese"),
    HumanMessage(content="I love programming in Python!"),
]

# Invoke the model with the messages
response = model.invoke(messages)
response.content


Use Output Parsers
The output of the model is more than just a string - it includes metadata. If we want to extract just the text of the translation, we can use an output parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parsed_result = parser.invoke(response)
parsed_result

Chaining Components Together
chain the model and the output parser together using the | operator

In [ ]:
chain = model | parser
translated_text = chain.invoke(messages)
translated_text

Using Prompt Templates for Translation

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


system_template = "Translate the following text into Japanese:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])


# Generate a structured message
result = prompt_template.invoke({"text": "I love programming in Python!"})
result.to_messages()

Chaining with LCEL (LangChain Expression Language)

We can now chain the prompt template, the language model, and the output parser to make the translation seamless

In [ ]:
chain = prompt_template | model | parser
final_translation = chain.invoke({"text": "I love programming in Python!"})
final_translation

Debugging with LangSmith

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API Key: ')

Deploying with LangServe


In [ ]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

import os
from langchain_openai import ChatOpenAI

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "Put your api here"

# Create the model instance
model = ChatOpenAI()

# Set up the components
system_template = "Translate the following text into Japanese:"Langsmithapi
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])
model = ChatOpenAI()
parser = StrOutputParser()

# Chain components
chain = prompt_template | model | parser

# FastAPI setup
app = FastAPI(title="LangChain English to Japanese Translation API", version="1.0")
add_routes(app, chain, path="/chain")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="localhost", port=8000)

Interacting Programmatically with the API

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
translated_text = remote_chain.invoke({"text": "I love programming in Python!"})
print(translated_text)